In [2]:
import pickle
import re
import string
import nltk
import pandas as pd
import spacy
import re
import functions as f
import string
from collections import Counter
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import TreebankWordTokenizer
# stemmerl = LancasterStemmer()
# stemmerp = PorterStemmer()
# stemmers = SnowballStemmer('english')
# stemmerwn = WordNetLemmatizer()
# spacy.prefer_gpu()

In [3]:
subs = ['wsb', 'science', 'ama', 'askreddit', 'worldnews', 'funny', 'dankmemes', 'memes', 'nosleep', 'psychology', 'politics', 'cscareerquestions', 'writingprompts']

# Load pickled files
data = {}
for i, c in enumerate(subs):
    with open("/home/nick/Documents/Reddit_NLP/pickles/" + c + ".pickle", "rb") as file:
        data[c] = pickle.load(file)

In [4]:
data.keys()
for key in data.keys():
    print(key + ': ' + str(len(data['{}'.format(key)])))

wsb: 36719
science: 59966
ama: 1885
askreddit: 101141
worldnews: 15265
funny: 6659
dankmemes: 4077
memes: 7332
nosleep: 6683
psychology: 1482
politics: 38821
cscareerquestions: 12703
writingprompts: 4026


In [5]:
data_combined = {key: [f.combine_text(value)] for (key, value) in data.items()}

In [6]:
pd.set_option('max_colwidth',150)

data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df.columns = ['text']
data_df

,text
wsb,In the future when Apple is at 50000 a share your gran kids will ask why you didn’t buy Apple at 400. And you’ll answer with “I bought corn instea...
science,"> The children were not motivated by achieving the maximum reward to the extent that adults were,” Blanco said. “Instead, children seemed primaril..."
ama,Tldr: do whatever the fuck you want. So you're telling me people on the internet LIE???? [removed] This is so funny this subreddit got fucked by o...
askreddit,"Victoria’s Secret was originally supposed to be a place for men to buy lingerie for their wives and not feel embarrassed, but now it’s almost excl..."
worldnews,"So was there any posts left after the purge? Ok, just for some context:\n\nFacebook has an estimated ~100 million fake user accounts.\nEvery month..."
funny,"But I wanted the robot ferret. Haha! Suck it you fools, I already got my robotic ferret from a mod! This not funny!\nRule number...... Oh, I forge..."
dankmemes,"Meme of the Month Seeding and Nominations for July 2020!\n\n# How do nominations work?\n\nTo nominate a meme, simply comment a name of a meme that..."
memes,"RedditMC is opening staff positions: Builders, Helpers (To become mod), Devs, GameMasters...\n\nMore infos [here](https://www.reddit.com/r/RedditM..."
nosleep,It looks like there may be more to this story. Click [here](https://www.reddit.com/message/compose/?to=UpdateMeBot&subject=Subscribe&message=Subsc...
psychology,"**\[Academic\] Decisions Regarding Mental Health Care (Canada, 17+)**\n\n**Study link**: [https://www.surveymonkey.ca/r/2VNC365](https://www.surve..."


Note: I am not lemmatizing for this part so that I can get accurate word counts for each subreddit. In the other notebooks I lemmatize the input.

In [7]:
clean = lambda x: f.clean(x)

data_clean = pd.DataFrame(data_df.text.apply(clean))
data_clean['subreddit'] = subs
data_clean

,text,subreddit
wsb,in the future when apple is at a share your gran kids will ask why you didnt buy apple at and youll answer with i bought corn instead you ever s...,wsb
science,the children were not motivated by achieving the maximum reward to the extent that adults were blanco said instead children seemed primarily moti...,science
ama,tldr do whatever the fuck you want so youre telling me people on the internet lie this is so funny this subreddit got fucked by one person a note...,ama
askreddit,victorias secret was originally supposed to be a place for men to buy lingerie for their wives and not feel embarrassed but now its almost exclusi...,askreddit
worldnews,so was there any posts left after the purge ok just for some context facebook has an estimated million fake user accounts every month about bil...,worldnews
funny,but i wanted the robot ferret haha suck it you fools i already got my robotic ferret from a mod this not funny rule number oh i forget carry on bu...,funny
dankmemes,meme of the month seeding and nominations for july how do nominations work to nominate a meme simply comment a name of a meme that started or ...,dankmemes
memes,redditmc is opening staff positions builders helpers to become mod devs gamemasters more infos using this to see if i have new flair cries in be...,memes
nosleep,it looks like there may be more to this story click to get a reminder to check back later got issues click httpswwwredditcomrnosleepwikinosleepau...,nosleep
psychology,decisions regarding mental health care canada study link you are invited to participate in a study named decisions regarding mental health ca...,psychology


In [8]:
cv = CountVectorizer()
data_cv = cv.fit_transform(data_clean.text)
dtm_cv = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
dtm_cv.index = data_clean.index
dtm_cv

,aa,aaa,aaaaa,aaaaaaaa,aaaaaaaaaaaa,aaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,...,𝘽𝙊𝘽,𝘽𝙐𝙄𝙇𝘿𝙀𝙍,𝘾𝘼𝙉,𝙁𝙄𝙓,𝙃𝙀,𝙄𝙏,𝙏𝙃𝙀,𝙔𝙀𝙎,𝚃𝚑𝚒𝚜,𝚝𝚎𝚖𝚙𝚕𝚊𝚝𝚎
wsb,0,5,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
science,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ama,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
askreddit,9,3,0,1,1,0,0,1,0,1,...,2,2,2,1,2,1,2,1,0,0
worldnews,2,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
funny,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dankmemes,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
memes,0,0,1,0,0,0,0,0,8,0,...,0,0,0,0,0,0,0,0,1,1
nosleep,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
psychology,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
tf_idf = TfidfVectorizer()
data_tf = tf_idf.fit_transform(data_clean.text)
dtm_idf = pd.DataFrame(data_tf.toarray(), columns=tf_idf.get_feature_names())
dtm_idf.index = data_clean.index
dtm_idf

,aa,aaa,aaaaa,aaaaaaaa,aaaaaaaaaaaa,aaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,...,𝘽𝙊𝘽,𝘽𝙐𝙄𝙇𝘿𝙀𝙍,𝘾𝘼𝙉,𝙁𝙄𝙓,𝙃𝙀,𝙄𝙏,𝙏𝙃𝙀,𝙔𝙀𝙎,𝚃𝚑𝚒𝚜,𝚝𝚎𝚖𝚙𝚕𝚊𝚝𝚎
wsb,0.000000,0.000354,0.000000,0.000000,0.000000,0.000000,0.000093,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
science,0.000105,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ama,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
askreddit,0.000084,0.000037,0.000000,0.000016,0.000014,0.000000,0.000000,0.000016,0.000000,0.000016,...,0.000033,0.000033,0.000033,0.000016,0.000033,0.000016,0.000033,0.000016,0.000000,0.000000
worldnews,0.000090,0.000000,0.000000,0.000000,0.000067,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
funny,0.000000,0.000000,0.000000,0.000000,0.000000,0.000408,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
dankmemes,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
memes,0.000000,0.000000,0.000548,0.000000,0.000000,0.000000,0.000000,0.000000,0.004383,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000548,0.000548
nosleep,0.000127,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
psychology,0.000652,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [15]:
# Find the number of unique words used in each subreddit

unique_list = []
dtm_cv = dtm_cv.transpose()
for sub in dtm_cv.columns:
    uniques = dtm_cv[sub].to_numpy().nonzero()[0].size
    unique_list.append(uniques)

data_words = pd.DataFrame(list(zip(subs, unique_list)), columns=['subreddit', 'unique_words'])
data_unique_sort = data_words.sort_values(by='unique_words', ascending=False)
data_unique_sort

,subreddit,unique_words
3,askreddit,55036
1,science,47700
10,politics,31929
12,writingprompts,26290
4,worldnews,21388
11,cscareerquestions,20344
0,wsb,19672
8,nosleep,11156
5,funny,10234
7,memes,8382


Somewhat interesting that askreddit and science have a similar number of unique words - at first I was thinking in terms of the 100,000 comments downloaded from askreddit and 50,000 comments downloaded from science - but then looking below, I realized that the total word counts of both subreddits are similar, both around 2,000,000. So the structure of comments in each subreddit is actually similar.

In [16]:
with open(f'/home/nick/Documents/Reddit_NLP/pickles/wsb.pickle', 'rb') as file:
    wsb = pickle.load(file)
with open('/home/nick/Documents/Reddit_NLP/pickles/science.pickle', 'rb') as file:
    science = pickle.load(file)
with open('/home/nick/Documents/Reddit_NLP/pickles/ama.pickle', 'rb') as file:
    ama = pickle.load(file)
with open('/home/nick/Documents/Reddit_NLP/pickles/askreddit.pickle', 'rb') as file:
    askreddit = pickle.load(file)
with open('/home/nick/Documents/Reddit_NLP/pickles/worldnews.pickle', 'rb') as file:
    worldnews = pickle.load(file)
with open('/home/nick/Documents/Reddit_NLP/pickles/funny.pickle', 'rb') as file:
    funny = pickle.load(file)
with open('/home/nick/Documents/Reddit_NLP/pickles/dankmemes.pickle', 'rb') as file:
    dankmemes = pickle.load(file)
with open('/home/nick/Documents/Reddit_NLP/pickles/politics.pickle', 'rb') as file:
    politics = pickle.load(file)
with open('/home/nick/Documents/Reddit_NLP/pickles/memes.pickle', 'rb') as file:
    memes = pickle.load(file)
with open('/home/nick/Documents/Reddit_NLP/pickles/psychology.pickle', 'rb') as file:
    psychology = pickle.load(file)
with open('/home/nick/Documents/Reddit_NLP/pickles/nosleep.pickle', 'rb') as file:
    nosleep = pickle.load(file)
with open('/home/nick/Documents/Reddit_NLP/pickles/cscareerquestions.pickle', 'rb') as file:
    cscareerquestions = pickle.load(file)

In [17]:
print('wsb: {}'.format(len(wsb)))
print('science: {}'.format(len(science)))
print('ama: {}'.format(len(ama)))
print('askreddit: {}'.format(len(askreddit)))    
print('worldnews: {}'.format(len(worldnews)))      
print('funny: {}'.format(len(funny)))      
print('dankmemes: {}'.format(len(dankmemes)))      
print('politics: {}'.format(len(politics)))      
print('psychology: {}'.format(len(psychology)))   
print('cscareerquestions: {}'.format(len(cscareerquestions)))   
print('nosleep: {}'.format(len(nosleep)))   
print('memes: {}'.format(len(memes)))   

wsb: 36719
science: 59966
ama: 1885
askreddit: 101141
worldnews: 15265
funny: 6659
dankmemes: 4077
politics: 38821
psychology: 1482
cscareerquestions: 12703
nosleep: 6683
memes: 7332


In [18]:
# Calculate the words per comment of each subreddit

# Find the total number of words that a subreddit uses
total_list = []
sub_name = []
for subreddit in dtm_cv.columns:
    totals = sum(dtm_cv[subreddit])
    sub_name.append(subreddit)
    total_list.append(totals)
    
data_words = pd.DataFrame(list(zip(dtm_cv.columns, total_list)), columns=['subreddit', 'total_words'])
data_total = data_words.sort_values(by='subreddit', ascending=True)
data_total

,subreddit,total_words
2,ama,39270
3,askreddit,2408194
11,cscareerquestions,682736
6,dankmemes,48463
5,funny,95399
7,memes,73906
8,nosleep,164738
10,politics,1091484
9,psychology,65849
1,science,2020170


In [23]:
num_dict = {'wsb': 36719, 'science': 59966, 'memes': 7332, 'cscareerquestions': 12703, 'politics': 38821, 'psychology': 1482, 'ama': 1885, 'dankmemes': 4077, 'worldnews': 15265, 'funny': 6659, 'nosleep': 6683, 'askreddit': 101141, 'writingprompts': 4026}
num_comments = [1885, 101141, 12703, 4077, 6659, 7332, 6683, 38821, 1482, 59966, 15265, 4026, 36719]

In [21]:
data_total['total_comments'] = num_comments
data_total['words_per_comment'] = (data_total['total_words'] / data_total['total_comments']).round(2)
data_total.sort_values(by='words_per_comment', ascending=False).reset_index(drop=True)

,subreddit,total_words,total_comments,words_per_comment
0,writingprompts,560048,4026,139.11
1,cscareerquestions,682736,12703,53.75
2,psychology,65849,1482,44.43
3,science,2020170,59966,33.69
4,worldnews,471033,15265,30.86
5,politics,1091484,38821,28.12
6,nosleep,164738,6683,24.65
7,askreddit,2408194,101141,23.81
8,ama,39270,1885,20.83
9,funny,95399,6659,14.33


These counts are higher than when I first calculated this - the first time, I semi-unknowingly filtered out stop words with CountVectorizer. However, even with the stop words included, the rankings are almost identical - the only difference is here, nosleep has more words per comment than askreddit. With stop words filtered out, askreddit has more comments. That means that nosleep has a ton of stop words in its comments.

Anyway, these numbers make sense. Writingprompts asks people to write creative stories in the comments. Cscareerquestions is for thorough, detailed questions. Science and psychology deliver factual information, as well as worldnews and politics to an extent. From there, the subreddits become preogressively less serious, until we reach memes, which has an average of 10 words per comment (that even seems a bit high for memes).